In [ ]:
from langgraph.graph import Graph, StateGraph
from typing import Dict, TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage, AIMessage

# 상태 타입 정의
class AgentState(TypedDict):
    messages: list[HumanMessage | AIMessage]
    input_type: str  # "text" 또는 "voice"
    need_voice: bool
    location: Dict[str, float] | None  # {"latitude": xx, "longitude": xx}
    intent: Literal["chat", "find_hospital", "find_pharmacy"] | None
    search_results: list[dict] | None

# 의도 분류 노드
async def classify_intent(state: AgentState) -> Literal["chat", "find_hospital", "find_pharmacy"]:
    """사용자 입력의 의도를 분류"""
    last_message = state["messages"][-1].content
    
    # GPT를 사용하여 의도 분류
    messages = [
        {"role": "system", "content": "Classify user intent into: chat, find_hospital, or find_pharmacy"},
        {"role": "user", "content": last_message}
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=50
    )
    intent = response.choices[0].message.content.strip().lower()
    
    return intent

# 위치 처리 노드
async def location_node(state: AgentState) -> AgentState:
    """위치 정보 처리"""
    if state["location"] is None and state["intent"] in ["find_hospital", "find_pharmacy"]:
        # 위치 정보 요청 메시지
        response = "위치 정보가 필요합니다. 주소나 지역명을 알려주세요."
        state["messages"].append(AIMessage(content=response))
    return state

# 병원 검색 노드
async def hospital_search_node(state: AgentState) -> AgentState:
    """주변 병원 검색"""
    if state["intent"] == "find_hospital" and state["location"]:
        # Google Places API나 공공데이터 API를 사용하여 병원 검색
        hospitals = search_nearby_hospitals(
            latitude=state["location"]["latitude"],
            longitude=state["location"]["longitude"]
        )
        state["search_results"] = hospitals
        
        # 검색 결과 포맷팅
        response = format_hospital_results(hospitals)
        state["messages"].append(AIMessage(content=response))
    return state

# 약국 검색 노드
async def pharmacy_search_node(state: AgentState) -> AgentState:
    """주변 약국 검색"""
    if state["intent"] == "find_pharmacy" and state["location"]:
        # 약국 검색 API 사용
        pharmacies = search_nearby_pharmacies(
            latitude=state["location"]["latitude"],
            longitude=state["location"]["longitude"]
        )
        state["search_results"] = pharmacies
        
        # 검색 결과 포맷팅
        response = format_pharmacy_results(pharmacies)
        state["messages"].append(AIMessage(content=response))
    return state

# 그래프 구성
workflow = Graph()

# 기본 노드 추가
workflow.add_node("stt", stt_node)
workflow.add_node("intent_classifier", classify_intent)
workflow.add_node("location", location_node)
workflow.add_node("hospital_search", hospital_search_node)
workflow.add_node("pharmacy_search", pharmacy_search_node)
workflow.add_node("llm", llm_node)
workflow.add_node("tts", tts_node)

# 조건부 라우팅
def router(state: AgentState) -> str:
    if state["intent"] == "find_hospital":
        return "hospital_search"
    elif state["intent"] == "find_pharmacy":
        return "pharmacy_search"
    else:
        return "llm"

# 엣지 연결
workflow.add_edge("stt", "intent_classifier")
workflow.add_edge("intent_classifier", "location")
workflow.add_edge("location", router)  # 조건부 라우팅
workflow.add_edge("hospital_search", "tts")
workflow.add_edge("pharmacy_search", "tts")
workflow.add_edge("llm", "tts")

# 컴파일
app = workflow.compile()

# 실행 예시
async def process_message(
    message: str,
    input_type: str = "text",
    need_voice: bool = False,
    location: Dict[str, float] | None = None
) -> AgentState:
    initial_state = AgentState(
        messages=[HumanMessage(content=message)],
        input_type=input_type,
        need_voice=need_voice,
        location=location,
        intent=None,
        search_results=None
    )
    
    final_state = await app.ainvoke(initial_state)
    return final_state